In [ ]:

#%%
import torch

args = {
    'epoch_num': 100,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'batch_size': 4,     # Mini-batch size. 600
    'batch_test': 4,     # size of test batch
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

dataset_name = "PAMAP2"
model_type = "AttentionTransformer"
#"OurConvLSTM", "AttentionTransformer", "DeepConvLSTM", "CnnIMU", FCNN
#NoISOurConvLSTM, ShuffleISOurConvLSTM, LstmISOur:ConvLSTM
val_sub = 4
for ts_sub in [5,6,7]: #2,6]:#[6,7,2,0,1,3,5]:


  ! pip install wget
  import os
  import torch
  import pandas as pd
  import numpy as np
  import torch
  from torch import nn


  ssh_config = """
  Host github.com
    IdentityFile ~/.ssh/github.pem
    User davipeag
    StrictHostKeyChecking no
  """

  if os.name == 'nt':
    base_path = ""
    REPO_DIR = "."
    STORE_DIR ="." 
    print("Windows")
  else:
    print("Unix-like")
    REPO_DIR = "/tmp/HeartRateRegression"
    from google.colab import drive
    drive.mount('/content/drive')
    GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
    DATA_PATH = "/content/drive/My\ Drive/deeplearning_project/normalized.zip"
    STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
    !mkdir ~/.ssh
    !cp -u {GIT_PATH} ~/.ssh/
    !chmod u=rw,g=,o= ~/.ssh/github.pem
    !echo "{ssh_config}" > ~/.ssh/config
    !chmod u=rw,g=,o= ~/.ssh/config
    ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
    ! (cd {REPO_DIR} && git pull )
    import sys
    sys.path.append(REPO_DIR)


  def git_push():
    if os.name == 'nt':
      pass
    else:
      ! git config --global user.email "daviaguiar@outlook.com"
      ! git config --global user.name "Davi Pedrosa de Aguiar"
      print("going to push")
      ! (cd {REPO_DIR} && git pull && cd -)
      ! (cd {REPO_DIR} && git add . && git commit -m "from colab" && git push)

  def git_pull():
    if os.name == 'nt':
      pass
    else:
      ! git config --global user.email "daviaguiar@outlook.com"
      ! git config --global user.name "Davi Pedrosa de Aguiar"
      print("going to push")
      ! (cd {REPO_DIR} && git pull && cd -)
      
    
  git_push()



  from data_utils import (
      Pamap2Handler, cross_validation_split)

  from default_utils import DefaultPamapPreprocessing, FcPamapPreprocessing
  from preprocessing_utils import (OurConvLstmToAttentionFormat, OurConvLstmToCnnImuFormat)

  from models_utils import OurConvLstmDataset, make_loader, reset_seeds
  from models_utils import DatasetXY

  from default_utils import TrainOurConvLSTM, TrainXY
  from default_utils import make_our_conv_lstm, make_attention_transormer_model, make_fcnn
  from torch import nn



  reset_seeds()

  ##%%

  dataset_handler = Pamap2Handler(os.path.join(REPO_DIR, ".."))

  dfs = [dataset_handler.get_protocol_subject(s) for s in [1,2,3,4,5,6,7,8,9]]
  #df_full = pd.concat(dfs)
  
  #use_dfs = dfs[:8]

  from preprocessing_utils import ShuffleIS

  preprocessing_options = {
      "OurConvLSTM": DefaultPamapPreprocessing(ts_count = 300, donwsampling_ratio = 1),
      "AttentionTransformer": DefaultPamapPreprocessing(
        last_transformer=OurConvLstmToAttentionFormat()),
      "DeepConvLSTM": DefaultPamapPreprocessing(last_transformer=OurConvLstmToCnnImuFormat()),
      "CnnIMU": DefaultPamapPreprocessing(last_transformer=OurConvLstmToCnnImuFormat()),
      "FCNN": FcPamapPreprocessing(),
      "NoISOurConvLSTM": DefaultPamapPreprocessing(ts_count = 300, donwsampling_ratio = 1),
      "ShuffleISOurConvLSTM": DefaultPamapPreprocessing(ts_count = 300, donwsampling_ratio = 1, last_transformer=ShuffleIS()),
      "LstmISOurConvLSTM": DefaultPamapPreprocessing(ts_count = 300, donwsampling_ratio = 1)
  }

  preprocessor =preprocessing_options[model_type]

  ##%%

  #preprocessor.transformers.fit(df_full)

  ###begin attempt
  train_ts = [0,1,2,3,4,5,6,7] 
  val_idx = train_ts.index(val_sub)
  ts_idx = train_ts.index(ts_sub)
  train_indices = [i for i in train_ts if i not in [val_idx, ts_idx]]# list(filter(lambda i: i not in [val_idx, ts_idx], train_ts))
  #train_idxes = train_ts[:val_idx] + train_ts[val_idx+1:] 

  xy_tr = [preprocessor.transformers.transform(dfs[i]) for i in train_indices]
  xy_val = [preprocessor.transformers_ts.transform(dfs[val_idx])]
  xy_ts = [preprocessor.transformers_ts.transform(dfs[ts_idx])]

  print(train_indices)
  print(ts_sub)


  #%% end attempt
  #xy_tr, xy_val, xy_ts = cross_validation_split(use_dfs, preprocessor.transformers, preprocessor.transformers_ts, preprocessor.transformers_ts, val_sub, ts_sub)

  del dfs
  #del df_full

  dataset_cls_options = {
      "OurConvLSTM": OurConvLstmDataset,
      "AttentionTransformer": DatasetXY,
      "DeepConvLSTM": DatasetXY,
      "CnnIMU": DatasetXY,
      "FCNN": DatasetXY,
      "NoISOurConvLSTM": OurConvLstmDataset,
      "ShuffleISOurConvLSTM": OurConvLstmDataset,
      "LstmISOurConvLSTM": OurConvLstmDataset,
  }


  dataset_cls = dataset_cls_options[model_type]

  loader_tr = make_loader(xy_tr, dataset_cls, batch_size=args["batch_size"], shuffle=True)
  loader_val = make_loader(xy_val, dataset_cls, batch_size=args["batch_test"],shuffle=False)
  loader_ts = make_loader(xy_ts, dataset_cls, batch_size=args["batch_test"], shuffle=False)


  ##%%
  from default_utils import make_cnn_imu2
  from default_utils import make_deep_conv_lstm


  net_options = {
      "OurConvLSTM": lambda : make_our_conv_lstm(40,1),
      "AttentionTransformer": lambda: make_attention_transormer_model(args["device"]),
      "DeepConvLSTM": lambda : make_deep_conv_lstm(),
      "CnnIMU": lambda : make_cnn_imu2(),
      "FCNN": lambda : make_fcnn(),
      "NoISOurConvLSTM":lambda : make_our_conv_lstm(40,1,"nois"),
      "ShuffleISOurConvLSTM": lambda : make_our_conv_lstm(40,1,"regular"),
      "LstmISOurConvLSTM": lambda : make_our_conv_lstm(40,1, "lstmis"),
  }

  reset_seeds()

  net = net_options[model_type]().to(args["device"])
  criterion = nn.L1Loss().to(args["device"]) 
  optimizer = torch.optim.Adam(net.parameters(), lr=args["lr"],
                              weight_decay=args["weight_decay"])

  ##%%


  basic_training_parameters = {
      "net": net,
      "criterion": criterion,
      "optimizer": optimizer,
      "loader_tr": loader_tr,
      "loader_val": loader_val,
      "loader_ts": loader_ts,
      "normdz": preprocessor.normdz,
      "ztransformer": preprocessor.ztransformer,
      "device": args["device"]
  }


  trainer_options = {
      "OurConvLSTM": lambda : TrainOurConvLSTM(**basic_training_parameters),
      "AttentionTransformer": lambda : TrainXY(
          **basic_training_parameters,
          get_last_y_from_x = lambda x: x[:,1, 0, -1].reshape(-1,1)
      ),
      "CnnIMU":lambda : TrainXY(
          **basic_training_parameters,
          get_last_y_from_x = lambda x: np.mean(x[:,0,200:300, 0], axis=1).reshape(-1,1)
      ),
      "DeepConvLSTM":lambda : TrainXY(
          **basic_training_parameters,
          get_last_y_from_x = lambda x: np.mean(x[:,0,200:300, 0], axis=1).reshape(-1,1)
      ),
      "FCNN":lambda : TrainXY(
          **basic_training_parameters,
          get_last_y_from_x = lambda x: x[:,0, 0].reshape(-1,1)
      ),
      "NoISOurConvLSTM":lambda : TrainOurConvLSTM(**basic_training_parameters),
      "ShuffleISOurConvLSTM": lambda : TrainOurConvLSTM(**basic_training_parameters),
      "LstmISOurConvLSTM": lambda : TrainOurConvLSTM(**basic_training_parameters),
  }

  trainer = trainer_options[model_type]()


  run_output = trainer.train_epochs(args["epoch_num"])


  #%%

  state_dict_name = f"trained_models/{model_type}ts_{ts_sub}_val_{val_sub}.pkl"
  torch.save(run_output["best_val_model"], os.path.join(STORE_DIR, state_dict_name))

  git_push()


# %%


cuda
Unix-like
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.ssh’: File exists
fatal: destination path 'HeartRateRegression' already exists and is not an empty directory.
Already up to date.
going to push
Already up to date.
/content
On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [ ]:
len(use_dfs)